In [8]:
# Basic models
import traceback
import numpy as np
import pandas as pd

# Data operation models
from sklearn.model_selection import StratifiedKFold
from sklearn import cross_validation   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV    #Performing grid search

# Classifier models
from xgboost.sklearn import XGBClassifier

# Evaluation models
import sklearn.metrics as sm


//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:44.156696. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [9]:
# Configurations for the model
def run_Config():
    data_name = 'Cardiac_Death_dummy.xlsx'
    k_holdout = 10
    k_cv = 10
    """pars = {
            'max_depth': [4,6,8,10],
            'min_child_weight': [1,2,3],
            'scale_pos_weight': [2,4,6]
            }"""
    # recommended values
    pars = {
            'max_depth': [6],
            'min_child_weight': [2],
            'scale_pos_weight': [2]
            }
    return data_name, k_holdout, k_cv, pars

//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:44.359087. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [10]:
# Load data and run the holdout cross validation
def run_load_data(data_name):
    all = pd.read_excel(data_name)
    all = all.values
    all_fea = all[:,:-1]
    all_label = get_normal_label(all[:,-1])
    return all_fea, all_label

def get_normal_label(y):
    y_uni = np.unique(np.array(y))
    for i in xrange(len(y_uni)):
        y[np.nonzero(y == y_uni[i])[0]] = i
    return y

def run_holdout_validation(all_fea, all_label, k_holdout, k_cv, pars):
    # holdout cv
    i_t = 0
    res_list = []
    opt_pars_list = []
    skf = StratifiedKFold(n_splits = k_holdout, shuffle = True)
    for i_train, i_test in skf.split(all_fea, all_label):
        # times
        i_t = i_t + 1
        print 'Round ', str(i_t), ' Holdout CV----------------------'
        # obtain current training and test data
        train_fea, test_fea = all_fea[i_train], all_fea[i_test]
        train_label, test_label = all_label[i_train], all_label[i_test]
        # learn the model
        pre_label_now, opt_pars_now = run_xgboost(pars, k_cv, train_fea, train_label, test_fea)
        # evaluate the prediction
        res_now = run_evaluation(pre_label_now, test_label)
        # save results
        res_list.append(res_now)
        opt_pars_list.append(opt_pars_now)
    return res_list, opt_pars_list
    

//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:44.547656. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [11]:
# Learn the model Xgboost
def run_xgboost(pars, k_cv, train_fea, train_label, test_fea):
    estimator = XGBClassifier()
    clf = GridSearchCV(estimator, pars, cv = k_cv)
    clf.fit(train_fea, train_label)
    pre_label = clf.predict(test_fea)
    print 'Xgboost is running...'
    return pre_label, clf.best_params_
    

//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:44.742471. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [12]:
# Evaluate the performance
def run_evaluation(p, y):
    res_dict = {}
    i_pos = np.nonzero(y == max(y))
    i_neg = np.nonzero(y == min(y))
    res_dict['TPR'] = (1 - sm.hamming_loss(y[i_pos], p[i_pos]))*100
    res_dict['TNR'] = (1 - sm.hamming_loss(y[i_neg], p[i_neg]))*100
    res_dict['MAcc'] = np.mean([res_dict['TPR'], res_dict['TNR']])
    res_dict['GM'] = np.sqrt(res_dict['TPR']*res_dict['TNR'])
    res_dict['F1(Macro)'] = sm.f1_score(y, p, average='macro')*100
    res_dict['F1(Micro)'] = sm.f1_score(y, p, average='micro')*100
    res_dict['Acc'] = sm.accuracy_score(y, p)*100
    return res_dict

//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:44.930256. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
# Print the results
def get_dict(d):
    list_final = []
    for i in d:
        list_now = i + '_' + str(d[i])
        list_final.append(list_now)
    return list_final

//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:45.117091. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [14]:
if __name__ == '__main__':
    data_name, k_holdout, k_cv, pars = run_Config()
    all_fea, all_label = run_load_data(data_name)
    res_list, opt_pars_list = run_holdout_validation(all_fea, all_label, k_holdout, k_cv, pars)
    print 'Each heldout cv result:'
    print '-----------------------'
    for i, j in zip(res_list,opt_pars_list):
        print get_dict(i), 'with hyper-parameters:'
        print get_dict(j)
        print '-----------------------'

Round  1  Holdout CV----------------------


//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_

Xgboost is running...
Round  2  Holdout CV----------------------


//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_

Xgboost is running...
Round  3  Holdout CV----------------------


...

//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_

Xgboost is running...
Round  10  Holdout CV----------------------


//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_

Xgboost is running...
Each heldout cv result:
-----------------------
['Acc_91.4285714286', 'F1(Micro)_91.4285714286', 'TNR_100.0', 'MAcc_62.5', 'TPR_25.0', 'F1(Macro)_67.6923076923', 'GM_50.0'] with hyper-parameters:
['scale_pos_weight_2', 'max_depth_6', 'min_child_weight_2']
-----------------------
['Acc_88.2352941176', 'F1(Micro)_88.2352941176', 'TNR_96.7741935484', 'MAcc_48.3870967742', 'TPR_0.0', 'F1(Macro)_46.875', 'GM_0.0'] with hyper-parameters:
['scale_pos_weight_2', 'max_depth_6', 'min_child_weight_2']
-----------------------
['Acc_91.1764705882', 'F1(Micro)_91.1764705882', 'TNR_100.0', 'MAcc_50.0', 'TPR_0.0', 'F1(Macro)_47.6923076923', 'GM_0.0'] with hyper-parameters:
['scale_pos_weight_2', 'max_depth_6', 'min_child_weight_2']
-----------------------
['Acc_88.2352941176', 'F1(Micro)_88.2352941176', 'TNR_96.7741935484', 'MAcc_48.3870967742', 'TPR_0.0', 'F1(Macro)_46.875', 'GM_0.0'] with hyper-parameters:
['scale_pos_weight_2', 'max_depth_6', 'min_child_weight_2']
------------

//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost/sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)
//anaconda/envs/tensorflow/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-07-25 14:36:45.307060. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
